In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import random
from datetime import datetime
# Import the time module.
import time
# Get today's date in seconds.
today = time.time()
# Import the API key.
from config import weather_api_key

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90., high=90., size=200)
lngs = np.random.uniform(low=-180., high=180., size=200)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

147

In [6]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=14d496981f3ab4ab4fbc15adae63ab96


In [14]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        print(city_lat)
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        #city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description
                         })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | aklavik
68.2191
Processing Record 2 of Set 1 | ushuaia
-54.8
Processing Record 3 of Set 1 | hilo
19.7297
Processing Record 4 of Set 1 | santa eulalia del rio
City not found. Skipping...
Processing Record 5 of Set 1 | hidalgotitlan
17.7833
Processing Record 6 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 7 of Set 1 | cape town
-33.9258
Processing Record 8 of Set 1 | galiwinku
City not found. Skipping...
Processing Record 9 of Set 1 | hasaki
35.7333
Processing Record 10 of Set 1 | luderitz
-26.6481
Processing Record 11 of Set 1 | airai
-8.9266
Processing Record 12 of Set 1 | attawapiskat
City not found. Skipping...
Processing Record 13 of Set 1 | leningradskiy
69.3833
Processing Record 14 of Set 1 | klaksvik
62.2266
Processing Record 15 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 16 of Set 1 | hambantota
6.1241
Processing Record 17 of Set 1 | yellowknif

In [19]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)[['City', 'Country', 'Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed', 'Current Description']]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Aklavik,CA,68.2191,-135.0107,34.63,100,90,16.11,light intensity drizzle
1,Ushuaia,AR,-54.8000,-68.3000,51.46,43,75,17.27,light rain
2,Hilo,US,19.7297,-155.0900,83.28,76,38,3.67,light rain
3,Hidalgotitlan,MX,17.7833,-94.6333,83.80,75,16,4.88,few clouds
4,Cape Town,ZA,-33.9258,18.4232,60.10,49,20,16.11,few clouds
5,Hasaki,JP,35.7333,140.8333,63.28,82,0,6.24,clear sky
6,Luderitz,NA,-26.6481,15.1594,63.95,46,0,28.77,clear sky
7,Airai,TL,-8.9266,125.4092,59.58,83,54,1.16,broken clouds
8,Leningradskiy,RU,69.3833,178.4167,15.24,84,76,4.90,broken clouds
9,Klaksvik,FO,62.2266,-6.5890,49.12,88,100,31.36,overcast clouds


In [16]:
city_data_df.to_csv('WeatherPy_Database.csv')